In [1]:
%pwd

'd:\\Major\\Medical-Chatbot-Generative-AI\\research'

In [2]:
import os 
os.chdir("../")

In [3]:
%pwd

'd:\\Major\\Medical-Chatbot-Generative-AI'

In [4]:
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

In [5]:
#Extract Data
def load_pdf_file(data):
    loader = DirectoryLoader(
        data,
        glob="*.pdf",
        loader_cls=PyPDFLoader
    )

    documents = loader.load()
    return documents

In [6]:
extracted_data=load_pdf_file(data='Data/')

In [7]:
extracted_data

[Document(metadata={'source': 'Data\\book.pdf', 'page': 0, 'page_label': '1'}, page_content=''),
 Document(metadata={'source': 'Data\\book.pdf', 'page': 1, 'page_label': '2'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'),
 Document(metadata={'source': 'Data\\book.pdf', 'page': 2, 'page_label': '3'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nA-B\n1'),
 Document(metadata={'source': 'Data\\book.pdf', 'page': 3, 'page_label': '4'}, page_content='STAFF\nJacqueline L. Longe, Project Editor\nDeirdre S. Blanchfield, Associate Editor\nChristine B. Jeryan, Managing Editor\nDonna Olendorf, Senior Editor\nStacey Blachford, Associate Editor\nKate Kretschmann, Melissa C. McDade, Ryan\nThomason, Assistant Editors\nMark Springer, Technical Specialist\nAndrea Lopeman, Programmer/Analyst\nBarbara J. Yarrow,Manager, Imaging and Multimedia\nContent\nRobyn V . Young,Project Manager

In [8]:
#Split the Data into Text Chunks
def text_split(extracted_data):
    text_splitter=RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks=text_splitter.split_documents(extracted_data)
    return text_chunks

In [9]:
text_chunks=text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 5860


In [10]:
from langchain.embeddings import HuggingFaceEmbeddings

In [11]:
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
    return embeddings

In [12]:
embeddings = download_hugging_face_embeddings()

C:\Users\SATHWIK\AppData\Local\Temp\ipykernel_8596\2604741814.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name='sentence-transformers/all-MiniLM-L6-v2')
c:\Users\SATHWIK\anaconda3\envs\medibot\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
query_result = embeddings. embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [14]:
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')

In [16]:
from pinecone import Pinecone, ServerlessSpec

In [17]:
pc = Pinecone(api_key=PINECONE_API_KEY)

In [18]:
index_name = "medicalbot"

pc.create_index(
    name=index_name,
    dimension=384, # Replace with your model dimensions
    metric="cosine", # Replace with your model metric
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    ) 
)

In [19]:
os.environ["PINECONE_API_KEY" ] = PINECONE_API_KEY

In [20]:
from langchain_pinecone import PineconeVectorStore
# . Embed each chunk .and upsert the embeddings into your Pinecone index.
docsearch = PineconeVectorStore. from_documents(
documents=text_chunks,
index_name=index_name,
embedding=embeddings,
)

In [21]:
# Load Existing index

from langchain_pinecone import PineconeVectorStore
# Embed each chunk and upsert the embeddings into your Pinecone index
docsearch = PineconeVectorStore. from_existing_index(
index_name=index_name,
embedding=embeddings
)

In [22]:
docsearch

In [23]:
retriever = docsearch. as_retriever(search_type="similarity", search_kwargs={"k":3})

In [24]:
retrieved_docs = retriever. invoke("What is Acne?")

In [25]:
retrieved_docs

[Document(id='3370ac3d-fe8f-4e45-b2d5-6814f47d1a69', metadata={'page': 39.0, 'page_label': '40', 'source': 'Data\\book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 226\nAcne\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 26'),
 Document(id='2583227e-546b-44ea-9224-74c5532b0192', metadata={'page': 38.0, 'page_label': '39', 'source': 'Data\\book.pdf'}, page_content='GALE ENCYCLOPEDIA OF MEDICINE 2 25\nAcne\nAcne vulgaris affecting a woman’s face. Acne is the general\nname given to a skin disorder in which the sebaceous\nglands become inflamed.(Photograph by Biophoto Associ-\nates, Photo Researchers, Inc. Reproduced by permission.)\nGEM - 0001 to 0432 - A  10/22/03 1:41 PM  Page 25'),
 Document(id='2f72947f-3ac7-476a-bb67-9e2ccdc57735', metadata={'page': 37.0, 'page_label': '38', 'source': 'Data\\book.pdf'}, page_content='Acidosis see Respiratory acidosis; Renal\ntubular acidosis; Metabolic acidosis\nAcne\nDefinition\nAcne is a common skin disease characterized by\npimples on the f

In [26]:
OPENAI_API_KEY=os.environ.get('OPENAI_API_KEY')

In [27]:
os.environ["OPENAI_API_KEY" ] = OPENAI_API_KEY

In [28]:
from langchain_openai import OpenAI
llm = OpenAI(temperature=0.4, max_tokens=500)

In [29]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.prompts import ChatPromptTemplate


system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer the question. "
    "If you don't know the answer, say that you don't know. "
    "Use three sentences maximum and keep the answer concise."
    "\n\n"
    "{context}"
)


# Corrected tuple format
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}")
    ]
)


In [30]:
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [31]:
response = rag_chain.invoke({"input": "What is Acne?"})
print(response["answer"])



Acne is a common skin disease characterized by pimples on the face, chest, and back. It occurs when the pores of the skin become clogged with oil, dead skin cells, and bacteria. Acne vulgaris is the most common type of acne, affecting approximately 17 million people in the United States. It can occur at any age, but is most common during adolescence.
